In [ ]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

# Import the usage count per program type data (from CSV)
data = pd.read_csv('Equip_usage_program_type2.csv') # Columns: Program_type, Equipment_Name, Usage_Count, Equipment_Status

# Filter data for only available equipment
#data = data[data['Equipment_Status'] == 'Available']  # Only keep rows where the equipment is available

# Create a list of unique program types and equipment names
program_types = data['Program_type'].unique().tolist()
equipment_names = data['Equipment_Name'].unique().tolist()

# Convert the data into a usable format (dictionary of usage counts)
usage_counts = {}
for _, row in data.iterrows():
    if row['Program_type'] not in usage_counts:
        usage_counts[row['Program_type']] = {}
    usage_counts[row['Program_type']][row['Equipment_Name']] = row['Usage_Count']

# Create the optimization model
model = gp.Model("Equipment Allocation")

# Create decision variables (binary variable x[i, j] means if equipment j is used in program i)
x = model.addVars(program_types, equipment_names, vtype=GRB.BINARY, name="x")

# Set the objective function to maximize the total usage count
model.setObjective(
    gp.quicksum(x[i, j] * usage_counts.get(i, {}).get(j, 0) for i in program_types for j in equipment_names),
    GRB.MAXIMIZE
)

# Add the constraints

# 1. Equipment availability constraint
# Ensure that no equipment is allocated more than once if its status is "Available"
for j in equipment_names:
    model.addConstr(
        gp.quicksum(x[i, j] for i in program_types) <= 1,  # An equipment can only be allocated to one program type per session
        name=f"Equipment_Availability_{j}"
    )

# 2. Program type capacity constraint
# Ensure that each program type doesn't exceed its available equipment usage
for i in program_types:
    model.addConstr(
        gp.quicksum(x[i, j] for j in equipment_names) <= len(equipment_names),  # The number of equipment assigned shouldn't exceed available options
        name=f"Program_Capacity_{i}"
    )

# Comment from Chuyun
# # Count the number of unique equipment for each Program_type
# max_equipment_per_program = data.groupby('Program_type')['Equipment_Name'].nunique().to_dict()
# for i in program_types:
#     model.addConstr(
#         gp.quicksum(x[i, j] for j in equipment_names) <= max_equipment_per_program[i],  # The number of equipment assigned shouldn't exceed available options
#         name=f"Program_Capacity_{i}"
#     )

# Optimize the model
model.optimize()

# Display results
if model.status == GRB.OPTIMAL:
    print("\nOptimal Equipment Allocation:")
    for i in program_types:
        for j in equipment_names:
            if x[i, j].x > 0.5:  # Only display allocated equipment
                print(f"Program Type: {i}, Equipment: {j}, Allocation: {x[i, j].x}")
else:
    print("No optimal solution found.")

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-20
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 23.4.0 23E214)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 17 rows, 42 columns and 84 nonzeros
Model fingerprint: 0x24b2c5c9
Variable types: 0 continuous, 42 integer (42 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 7e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 49.0000000
Presolve removed 17 rows and 42 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 73 49 

Optimal solution found (tolerance 1.00e-04)
Best objective 7.300000000000e+01, best bound 7.300000000000e+01, gap 0.0000%

Optima